In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [ ]:
var tmpDB = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1");

In [ ]:
tmpDB.Sessions

In [ ]:
var Fails = tmpDB.Sessions.Where((ISessionInfo si) => !si.SuccessfulTermination).ToArray();
Fails

In [ ]:
Fails[0].GetSessionDirectory()

In [ ]:
Fails[0].DeployPath

In [ ]:
tmpDB.Sessions.Pick(0).Actions()

In [ ]:
//tmpDB.Sessions.Where(s=>!s.SuccessfulTermination).ForEach(s=>s.ProjectName="SIM_Multicore_HHLR_overtime")

In [ ]:
string WFlowName = "run0";
WorkflowMgm.Init(WFlowName);
WorkflowMgm.AllJobs;

In [ ]:
WorkflowMgm.Sessions

In [ ]:
WorkflowMgm.SessionTable.Rows.Count;

In [ ]:
using System.IO;

In [ ]:
DirectoryInfo dir = Directory.CreateDirectory(CurrentDocDir+@"\"+WFlowName);
string dirpath    = dir.FullName;

In [ ]:
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("SipMatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}

In [ ]:
WorkflowMgm.AdditionalSessionTableColums.Clear();
WorkflowMgm.AdditionalSessionTableColums.Add(
    "MatrixAssembly", AddCols.XdgMatrixAssembly_time);
WorkflowMgm.AdditionalSessionTableColums.Add(
    "Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
WorkflowMgm.AdditionalSessionTableColums.Add(
    "Solver_Init_time", AddCols.Solver_Init_time);
WorkflowMgm.AdditionalSessionTableColums.Add(
    "Solver_Run_time", AddCols.Solver_Run_time);
WorkflowMgm.AdditionalSessionTableColums.Add(
    "NoOfCores", AddCols.NoOfCores);

In [ ]:
var SessTab = WorkflowMgm.SessionTable;

In [ ]:
SessTab.GetColumnNames().OrderBy(s=>s);

In [ ]:
// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = SessTab.GetColumnNames().ToList();
AllCols.Remove("Session");

In [ ]:
SessTab = SessTab.ExtractColumns(AllCols.ToArray());

In [ ]:
// Filename
var now           = DateTime.Now;
SessTab.TableName = WFlowName + "_" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath    = Path.Combine(dirpath, SessTab.TableName + ".json");

In [ ]:
SessTab.SaveToFile(docpath);

In [ ]:
docpath

In [ ]:
var CleanTab = SessTab.ExtractRows(delegate(int i, IDictionary<string,object> row) {
    return (Convert.ToBoolean(row["RegularTerminated"]) == true)
          && (Convert.ToInt32(row["Conv"]) != 0);
    });

In [ ]:
var SubTab = CleanTab.ExtractColumns("SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
"Aggregation_basis_init_time",
"Solver_Init_time",
"Solver_Run_time", "NoIter");

In [ ]:
SubTab.Print();

In [ ]:
SubTab.ToCSVFile(dirpath+@"\"+WFlowName+"_scaling.txt");

In [ ]:
var SubTab2 = CleanTab.ExtractColumns("DGdegree:u", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", 
              "minSolRunT","UsedNoOfMultigridLevels",
              "SipMatrixAssembly", "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time");

In [ ]:
//var SubTab2 = CleanTab.ExtractColumns("DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs","minSolRunT","SipMatrixAssembly", "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time");

In [ ]:
SubTab2.Print();

In [ ]:
SubTab2.ToCSVFile(dirpath+@"\"+WFlowName+"_profiling.txt");